# 1) Reading in  and Cleaning the Data

In [359]:
import os
import pandas as pd
import numpy as np
os.chdir('/Users/mariomoreno/Desktop/ML Projects/Mexico OC/data')

In [360]:
industria_cols = ['entidad', 'prod_oro_kg', 'prod_plata_kg', 'del', 'prod_plomo_t', 'prod_cobre_t', 'prod_zinc_t', 'del', 'prod_coque_t', 'del', 'prod_fierropellets_t', 'del', 'prod_azufre_t', 'del', 'prod_barita_t', 'prod_fluorita_t']
medio_cols = ['del', 'entidad', 'capital', 'altitud', 'costa_km', 'municipios', 'del']
seguridad_cols = ['entidad', 'denuncias_total', 'denuncias_salud', 'denuncias_arma', 'denuncias_bancaria', 'denuncias_leyg', 'denuncias_comms', 'del', 'denuncias_servpub', 'denuncias_fiscales', 'denuncias_ambiente', 'denuncias_asocdelictuosa', 'denuncias_patrimoniales', 'denuncias_propintelectual', 'denuncias_robocarretera', 'denuncias_otras']
carretera_cols = ['entidad', 'total_carretera_km', 'del', 'carretera_pavimentada_km', 'del', 'carretera_revestida_km', 'carretera_terraceria_kg', 'brechas_mejoradas_km', 'del']
educacion_cols = ['entidad', 'alumnos_inscritos_hs', 'hombres_inscritos_hs', 'mujeres_inscritas_hs', 'maestros_hs', 'escuelas_hs', 'del']
trabajo_cols = ['entidad', 'convenios_trabajo_no_juicio', 'conflictos_trabajo', 'emplazamientos_huelga', 'emplazamientos_huelga_solucionados', 'huelgas_estalladas', 'huelgas_locales_solucionadas', 'del']

Here I'm just simply renaming the columns to make it easier to delete unnecessary ones down the road.

In [361]:
medio_base = pd.read_excel('Medio_Ambiente.xlsx', sheetname = '1.11', header = 3, names = medio_cols)
seguridad_base = pd.read_excel('Seguridad.xlsx', sheetname = '7.2', header = 4, names = seguridad_cols)
carretera_base = pd.read_excel('Carretera.xlsx', sheetname = '14.2', header = 5, names = carretera_cols)
educacion_base = pd.read_excel('Educacion.xlsx', sheetname = '4.19', header = 4, names = educacion_cols)
industria_base = pd.read_excel('Industria.xlsx', sheetname = '11.5', header = 7, names = industria_cols)
trabajo_base = pd.read_excel('Trabajo.xlsx', sheetname = '8.15', header = 3, names = trabajo_cols)

Reading in the data, with appropriate sheetnames, headers in the right location, and appropriate column names as outlined above...though come to think of it, did not need headers if column names were changing.

In [362]:
base_frames = [medio_base, seguridad_base, carretera_base, educacion_base, industria_base, trabajo_base]
index_list = []
for frame in base_frames:
    frame.drop('del', axis =1, inplace=True)
    for ind, row in frame.iterrows():
        if row['entidad'] == 2010 or row['entidad'] == '2010/2011':
            print(ind)

4
543
543
408
560


Dropping all the empty columns, getting index numbers to cut frames starting at 2010 since I can't figure out how to do it otherwise

In [363]:
seguridad_base = seguridad_base.truncate(4).dropna(axis=0, how='all').reset_index(drop=True)
carretera_base = carretera_base.truncate(543).dropna(axis=0, how='all').reset_index(drop=True)
educacion_base = educacion_base.truncate(543).dropna(axis=0, how='all').reset_index(drop=True)
industria_base = industria_base.truncate(408).dropna(axis=0, how='all').reset_index(drop=True)
trabajo_base = trabajo_base.truncate(560).dropna(axis=0, how='all').reset_index(drop=True)
medio_base = medio_base.dropna(axis=0, how='all').reset_index(drop=True)

In [364]:
base_frames = [medio_base, seguridad_base, carretera_base, educacion_base, industria_base, trabajo_base]
for frame in base_frames:
    test = []
    for ind, row in frame.iterrows():
        if type(row['entidad']) is int or row['entidad'] == '2016 P/' or row['entidad'] == '2017 P/':
            test.append((row['entidad'], ind))
        elif row['entidad'] == '2010/2011' or row['entidad'] == '2011/2012' or row['entidad'] == '2012/2013' or row['entidad'] == '2013/2014' or row['entidad'] == '2014/2015' or row['entidad'] == '2015/2016' or row['entidad'] == '2016/2017':
            test.append((row['entidad'], ind))
    print(test)

[]
[(2010, 0), (2011, 34), (2012, 69), (2013, 104), (2014, 140), (2015, 175), ('2016 P/', 211)]
[(2010, 0), (2011, 34), (2012, 68), (2013, 102), (2014, 136), (2015, 170), ('2016 P/', 204), ('2017 P/', 238)]
[('2010/2011', 0), ('2011/2012', 34), ('2013/2014', 102), ('2014/2015', 136), ('2015/2016', 170)]
[(2010, 0), (2011, 24), (2012, 48), (2013, 72), (2014, 96), (2016, 144), ('2017 P/', 168)]
[(2010, 0), (2011, 35), (2012, 70), (2013, 105), (2014, 140), (2015, 175), (2016, 210), (2017, 245)]


Ok, so after truncating from 2010 onwards, dropping all rows that were just NAs, and reseting the indices, I know have indices where each year begins. This is useful because it let's me see that I only have complete data through 2016 and it let's me start thinking about how I can create a column for year across all datasets

In [386]:
carretera_base = carretera_base[:238]
educacion_base = educacion_base[:238]
industria_base = industria_base[:168]
trabajo_base = trabajo_base[:245]

In [406]:
# Trabajo - year column
tra = trabajo_base['entidad'].tolist()
year_tra = []
for ind, ent in enumerate(tra):
    if ind >= 0 and ind < 35:
        ent = 2010
        year_tra.append(ent)
    elif ind >= 35 and ind < 70:
        ent = 2011
        year_tra.append(ent)
    elif ind >= 70 and ind < 105:
        ent = 2012
        year_tra.append(ent)
    elif ind >= 105 and ind < 140:
        ent = 2013
        year_tra.append(ent)
    elif ind >= 140 and ind < 175:
        ent = 2014
        year_tra.append(ent)
    elif ind >= 175 and ind < 210:
        ent = 2015
        year_tra.append(ent)
    else:
        ent = 2016
        year_tra.append(ent)

tcol = pd.Series(year_tra)
trabajo_base['year'] = tcol.values

# Industria - year column
inds = industria_base['entidad'].tolist()
year_ind = []
for ind, ent in enumerate(inds):
    if ind >= 0 and ind < 24:
        ent = 2010
        year_ind.append(ent)
    elif ind >= 24 and ind < 48:
        ent = 2011
        year_ind.append(ent)
    elif ind >= 48 and ind < 72:
        ent = 2012
        year_ind.append(ent)
    elif ind >= 72 and ind < 96:
        ent = 2013
        year_ind.append(ent)
    elif ind >= 96 and ind < 120:
        ent = 2014
        year_ind.append(ent)
    elif ind >= 120 and ind < 144:
        ent = 2015
        year_ind.append(ent)
    else:
        ent = 2016
        year_ind.append(ent)

icol = pd.Series(year_ind)
industria_base['year'] = icol.values

# Educacion - year column
edu = educacion_base['entidad'].tolist()
year_ed = []
for ind, ent in enumerate(edu):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_ed.append(ent)
    elif ind >= 34 and ind < 68:
        ent = 2011
        year_ed.append(ent)
    elif ind >= 68 and ind < 102:
        ent = 2012
        year_ed.append(ent)
    elif ind >= 102 and ind < 136:
        ent = 2013
        year_ed.append(ent)
    elif ind >= 136 and ind < 170:
        ent = 2014
        year_ed.append(ent)
    elif ind >= 170 and ind < 204:
        ent = 2015
        year_ed.append(ent)
    else:
        ent = 2016
        year_ed.append(ent)

ecol = pd.Series(year_ed)
educacion_base['year'] = ecol.values

# Carretera - year column
car = carretera_base['entidad'].tolist()
year_car = []
for ind, ent in enumerate(car):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_car.append(ent)
    elif ind >= 34 and ind < 68:
        ent = 2011
        year_car.append(ent)
    elif ind >= 68 and ind < 102:
        ent = 2012
        year_car.append(ent)
    elif ind >= 104 and ind < 136:
        ent = 2013
        year_car.append(ent)
    elif ind >= 136 and ind < 170:
        ent = 2014
        year_car.append(ent)
    elif ind >= 170 and ind < 204:
        ent = 2015
        year_car.append(ent)
    else:
        ent = 2016
        year_car.append(ent)

ccol = pd.Series(year_car)
carretera_base['year'] = ccol.values

# Seguridad - year column
seg = seguridad_base['entidad'].tolist()
year_seg = []
for ind, ent in enumerate(seg):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_seg.append(ent)
    elif ind >= 34 and ind < 69:
        ent = 2011
        year_seg.append(ent)
    elif ind >= 69 and ind < 104:
        ent = 2012
        year_seg.append(ent)
    elif ind >= 104 and ind < 140:
        ent = 2013
        year_seg.append(ent)
    elif ind >= 140 and ind < 175:
        ent = 2014
        year_seg.append(ent)
    elif ind >= 175 and ind < 211:
        ent = 2015
        year_seg.append(ent)
    else:
        ent = 2016
        year_seg.append(ent)

scol = pd.Series(year_seg)
seguridad_base['year'] = scol.values

/Users/mariomoreno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [434]:
new_df = pd.merge(educacion_base, trabajo_base, how = 'outer', on=['entidad','year'])
new_df2 = pd.merge(new_df, seguridad_base, how = 'outer', on=['entidad', 'year'])
new_df3 = pd.merge(new_df2, carretera_base, how = 'outer', on = ['entidad', 'year'])
almost = pd.merge(new_df3, industria_base, how = 'outer', on = ['entidad', 'year'])

In [435]:
almost

,entidad,alumnos_inscritos_hs,hombres_inscritos_hs,mujeres_inscritas_hs,maestros_hs,escuelas_hs,year,convenios_trabajo_no_juicio,conflictos_trabajo,emplazamientos_huelga,...,prod_oro_kg,prod_plata_kg,prod_plomo_t,prod_cobre_t,prod_zinc_t,prod_coque_t,prod_fierropellets_t,prod_azufre_t,prod_barita_t,prod_fluorita_t
0,2010/2011,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estados Unidos Mexicanos,3811473,1858644,1952829,250712.0,13711,2010,261318.0,197733.0,36097.0,...,79375.5,4410749.0,192062.0,270136.0,570004.0,2105000.0,7931194.0,991802.0,143225.0,1067386.0
2,Aguascalientes,40129,19250,20879,2727.0,144,2010,2698.0,3239.0,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Baja California,107624,53692,53932,6192.0,271,2010,2342.0,9353.0,3241.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Baja California Sur,23247,11780,11467,1588.0,76,2010,1255.0,1660.0,296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Campeche,28350,14349,14001,1975.0,106,2010,3373.0,1179.0,144.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Coahuila de Zaragoza,82553,41397,41156,6219.0,330,2010,23331.0,7867.0,545.0,...,0,122602.0,0.0,0.0,0.0,1648709.0,2567865.0,0.0,22161.0,121833.0
7,Colima,23654,11567,12087,1528.0,91,2010,1962.0,935.0,233.0,...,0,0.0,0.0,0.0,0.0,0.0,3883423.0,0.0,0.0,0.0
8,Chiapas,183881,95643,88238,9449.0,704,2010,4008.0,1584.0,192.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,236841.0,0.0,0.0
9,Chihuahua,114630,55629,59001,6405.0,457,2010,10818.0,9810.0,246.0,...,18256.6,783081.0,46308.0,13132.0,133734.0,0.0,0.0,0.0,0.0,0.0


Outside of a few things, like merge not matching up Oaxaca and Coahuila in educacion and seguridad, this is pretty close to the final dataframe with a few edits still pending. 

In [436]:
almost.columns

Index(['entidad', 'alumnos_inscritos_hs', 'hombres_inscritos_hs',
       'mujeres_inscritas_hs', 'maestros_hs', 'escuelas_hs', 'year',
       'convenios_trabajo_no_juicio', 'conflictos_trabajo',
       'emplazamientos_huelga', 'emplazamientos_huelga_solucionados',
       'huelgas_estalladas', 'huelgas_locales_solucionadas', 'denuncias_total',
       'denuncias_salud', 'denuncias_arma', 'denuncias_bancaria',
       'denuncias_leyg', 'denuncias_comms', 'denuncias_servpub',
       'denuncias_fiscales', 'denuncias_ambiente', 'denuncias_asocdelictuosa',
       'denuncias_patrimoniales', 'denuncias_propintelectual',
       'denuncias_robocarretera', 'denuncias_otras', 'total_carretera_km',
       'carretera_pavimentada_km', 'carretera_revestida_km',
       'carretera_terraceria_kg', 'brechas_mejoradas_km', 'prod_oro_kg',
       'prod_plata_kg', 'prod_plomo_t', 'prod_cobre_t', 'prod_zinc_t',
       'prod_coque_t', 'prod_fierropellets_t', 'prod_azufre_t',
       'prod_barita_t', 'prod_fluorita